# Empty values

This page focuses on handling empty values in SQL.

The next cell creates a docker container with postgres, which will be used for the experiments on this page.

In [6]:
docker run --rm -d\
    -e POSTGRES_PASSWORD=postgres \
    --name empty_value_pg \
    postgres:15.4
docker run --rm -itd --name empty_value_sqlite keinos/sqlite3
sleep 5

33cbe8feb990bbe6afea7d3e7d6c3cc6cf7959251d2353cb496469cda2e37a93
de0267939e94b797d7ad1e87810300c77a8ca6f58dcb1225e52c3eb939409fec


**Note** Don't forget to close the container when you have finished playing with the examples.

In [ ]:
docker stop empty_value_pg empty_value_sqlite

## Replace in query

It's a very common case when empty values have business sense, but in query results you need to replace them. So you can write `SELECT` to replace `NULL` in query results, this section is focused on it.

### COALESCE

In PostgreSQL, there is the `COALESCE` function. Use the syntax `COALESCE(<column_name> | <value>, ...)`. This function will return the first value that is not `NULL`. You can use columns or just values as arguments—values will be broadcast to the column size.

Check the [description of the `COALESCE`](https://www.postgresql.org/docs/current/functions-conditional.html#FUNCTIONS-COALESCE-NVL-IFNULL) function in the official PostgreSQL documentation.

---

The following cell shows applying `COALENCE` to the columns `col1`, `col2` and `col3` that contains many empty values.

In [9]:
docker exec -i empty_value_example psql -U postgres << EOF
SELECT 
    *,
    COALESCE(col1, col2, col3) no_default,
    COALESCE(col1, col2, col3) IS NULL no_default_is_null,
    COALESCE(col1, col2, col3, 'default') with_default
FROM (
    VALUES
    (NULL, 'a', NULL),
    ('b', NULL, NULL),
    (NULL, NULL, 'c'),
    (NULL, NULL, NULL)

) AS temp(col1, col2, col3);
EOF

 col1 | col2 | col3 | no_default | no_default_is_null | with_default 
------+------+------+------------+--------------------+--------------
      | a    |      | a          | f                  | a
 b    |      |      | b          | f                  | b
      |      | c    | c          | f                  | c
      |      |      |            | t                  | default
(4 rows)



The same example in `sqlite`:

In [21]:
docker exec empty_value_sqlite sqlite3 -box -cmd "
CREATE TABLE IF NOT EXISTS tab (
    col1 TEXT, col2 TEXT, col3 TEXT
);


INSERT INTO tab (col1, col2, col3) VALUES
(NULL, 'a', NULL),
('b', NULL, NULL),
(NULL, NULL, 'c'),
(NULL, NULL, NULL);


SELECT 
    *,
    COALESCE(col1, col2, col3) no_default,
    COALESCE(col1, col2, col3) IS NULL no_default_is_null,
    COALESCE(col1, col2, col3, 'default') with_default
FROM tab;
"

┌──────┬──────┬──────┬────────────┬────────────────────┬──────────────┐
│ col1 │ col2 │ col3 │ no_default │ no_default_is_null │ with_default │
├──────┼──────┼──────┼────────────┼────────────────────┼──────────────┤
│      │ a    │      │ a          │ 0                  │ a            │
│ b    │      │      │ b          │ 0                  │ b            │
│      │      │ c    │ c          │ 0                  │ c            │
│      │      │      │            │ 1                  │ default      │
└──────┴──────┴──────┴────────────┴────────────────────┴──────────────┘


### Alternative (IFNULL)

IFNULL is an alternative function that can be used to replace empty values in query statements.

It's not supported by postgres. So to make sure, I'll show you in the following cell that there will be an error if you try to use it in postgres.

So I will use sqlite base to demonstrate how IFNULL works. In the following example is the process of creating sample database.

In [16]:
docker exec empty_value_sqlite sqlite3 -box -cmd "
CREATE TABLE IF NOT EXISTS tab (
    var1 TEXT,
    var2 TEXT
);


INSERT INTO tab (var1, var2) VALUES
(NULL, 'a'),
('b', NULL),
(NULL, NULL);


SELECT 
    COALESCE(var2, var1, 'relaced by COALESCE') coalesce,
    IFNULL(var1, var2)
FROM tab;
"

┌─────────────────────┬────────────────────┐
│      coalesce       │ IFNULL(var1, var2) │
├─────────────────────┼────────────────────┤
│ a                   │ a                  │
│ b                   │ b                  │
│ relaced by COALESCE │                    │
└─────────────────────┴────────────────────┘


The syntax is exactly the same as for `COALESCE`: `IFNULL(<column name>, <value to replace>)`.

Actually, sqlite supports both `COALESCE` and `IFNULL`, so in the following example both have been used to replace empty values in different columns.

In [15]:
%%bash
sqlite3 fill_empty_files/dump.sql

SELECT 
    IFNULL(var1, 'replaced by IFNULL') var1,
    COALESCE(var2, 'relaced by COALESCE') var2
FROM tab;

replaced by IFNULL|world
hello|relaced by COALESCE
replaced by IFNULL|test line


## Change base

Sometimes empty values in database appear as errors or business sense of data table can be selected. So you can face with case when you need to replace all values in the database, not only in the results of the specific query.

For such purpose you can use syntax:

```
UPDATE <table_name>
SET <column name> = <replace value>
WHERE <column name> IS NULL;
```

The following cell uses such syntax to replace some empty values.

In [17]:
%%bash
docker exec -i fill_empty psql -U postgres -d postgres

UPDATE tab
SET var1 = 'replaced'
WHERE var1 IS NULL;

UPDATE 2


Let's see the result of the transformation.

In [18]:
%%bash
docker exec -i fill_empty psql -U postgres -d postgres
SELECT * FROM tab;

   var1   |   var2    
----------+-----------
 hello    | 
 replaced | world
 replaced | test line
(3 rows)

